In [40]:
import numpy as np
import pandas as pd 
import csv

In [37]:
feature_column = ['f'+str(i) for i in range(1, 2049)]
out_colum_name = ['business_id'] + feature_column 

In [38]:
def getOneRow(name, grouped_df, train=1):
    out_list = [name]
    for f in feature_column:
        avg = np.mean(grouped_df[f])
        out_list.append(avg)
    return out_list

In [6]:
data_root = '/Users/atulkumar/ml/yelp/'
train_label = pd.read_csv(data_root+'train.csv')
train_photo_to_biz = pd.read_csv(data_root+'train_photo_to_biz_ids.csv')
train_df = pd.read_csv(data_root+"train_features.csv", header=None)
train_df.columns = ['photo_id'] +  feature_column
train_df1 = pd.merge(train_photo_to_biz, train_df, how='inner', on='photo_id')
train_df2 = train_df1.groupby('business_id')

In [42]:
train_out_handle = open('train_business_feature.csv', "w")
train_writer = csv.writer(train_out_handle)
train_writer.writerow(out_colum_name)
counter = 0
for name, group in train_df2:
    out_row = getOneRow(name, group, train=1)
    assert len(out_row) == len(out_colum_name)
    train_writer.writerow(out_row)
    counter += 1
    if counter%10000 == 0:
        print counter
train_out_handle.close()

In [47]:
train_business_feature = pd.read_csv(data_root+'train_business_feature.csv')
train_df3 = pd.merge(train_business_feature, train_label, how='inner', on='business_id')

In [132]:
train_df3['labels'].fillna('', inplace=True)
y_train = np.array([y.split() for y in train_df3['labels']])
y_train = [map(int, y) for y in y_train]

In [ ]:
from sklearn import svm, datasets
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import label_binarize
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
y_ptrain_mlb= mlb.fit_transform(y_train) 

random_state = np.random.RandomState(0)
X_ptrain, X_ptest, y_ptrain, y_ptest = \
    train_test_split(train_business_feature, y_ptrain_mlb, 
                 test_size=.2,random_state=random_state)
    
classifier = OneVsRestClassifier(svm.SVC(kernel='linear', probability=True))
classifier.fit(X_ptrain, y_ptrain)

y_ppredict = classifier.predict(X_ptest)


In [ ]:
from sklearn.metrics import f1_score

print "F1 score: ", f1_score(y_ptest, y_ppredict, average='micro') 
print "Individual Class F1 score: ", f1_score(y_ptest, y_ppredict, average=None)